# Attempts to speed up Voronoi-binning

Isaac Cheng - November 2021


In [131]:
import time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import reproject
from astropy.io import fits
from astropy.wcs import WCS
from vorbin.voronoi_2d_binning import voronoi_2d_binning
# My packages
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import plot_utils as pu
import fast_vorbin as fv
# Reload my packages
import importlib
importlib.reload(fu)
importlib.reload(pu)
importlib.reload(fv)

<module 'fast_vorbin' from '/arc/home/IsaacCheng/coop_f2021/packages/fast_vorbin.py'>

In [128]:
# 
# Explicitly setting array values to floats
# 
# Generate uniform pixels
num_px = 100
x_coords = np.arange(1, num_px + 1).astype(int)
y_coords = np.arange(1, num_px + 1).astype(int)
x_coords, y_coords = np.meshgrid(x_coords, y_coords)
gi_colour = np.full(x_coords.size, 5.0)
gi_colour_err = np.full(x_coords.size, 1.0)
# 
starttime = time.time()
binNum, xNode, yNode, xBar, yBar, sn, nPixels, scale = fv.voronoi_2d_binning(
    y_coords.flatten(),
    x_coords.flatten(),
    gi_colour.flatten(),
    gi_colour_err.flatten(),
    targetSN=10,
    pixelsize=1,
    cvt=False,
    quiet=True,
    wvt=False,
    sn_func=None,
)
endtime = time.time()
timediff = endtime - starttime
print("It took", timediff, "seconds to complete Voronoi binning.")


cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
fig, ax = plt.subplots()
img = ax.scatter(
    x_coords.flatten(), y_coords.flatten(), c=(gi_colour/gi_colour_err), cmap=cmap, s=1
)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Original Data (all 5.0)")
ax.set_title("Random data for debugging\\ldots")
# ax.grid(False)
ax.set_aspect("equal")
# fig.savefig("imgs/raw_random_data.pdf")
plt.show()

currentBin type <class 'numpy.int64'>
index 1
currentBin 0
xBar, xBar.shape, x[unBinned].shape 1.0 () (9999,)
nextBin type <class 'numpy.ndarray'>
nextBin dtype int64
SN type <class 'numpy.float64'>
xBar, yBar: 1.0 1.5
xBar, xBar.shape, x[unBinned].shape 1.0 () (9998,)
nextBin type <class 'numpy.ndarray'>
nextBin dtype int64
SN type <class 'numpy.float64'>
xBar, yBar: 1.3333333333333333 1.3333333333333333
xBar, xBar.shape, x[unBinned].shape 1.3333333333333333 () (9997,)
nextBin type <class 'numpy.ndarray'>
nextBin dtype int64
SN type <class 'numpy.float64'>
xBar, yBar: 1.5 1.5
xBar, xBar.shape, x[unBinned].shape 1.5 () (9996,)
nextBin type <class 'numpy.ndarray'>
nextBin dtype int64
SN type <class 'numpy.float64'>
index 2
currentBin 2
xBar, xBar.shape, x[unBinned].shape 1.0 () (9995,)
nextBin type <class 'numpy.ndarray'>
nextBin dtype int64
SN type <class 'numpy.float64'>
xBar, yBar: 1.0 3.5
xBar, xBar.shape, x[unBinned].shape 1.0 () (9994,)
nextBin type <class 'numpy.ndarray'>
nextBin

KeyboardInterrupt: 

In [16]:
import numpy as np
from numba_kdtree import KDTree
data = np.random.random(3000000).reshape(-1, 3)
print(data.shape)
kdtree = KDTree(data, leafsize=16)

# query the nearest neighbors of the first 100 points
distances, indices = kdtree.query(data[:100], k=30)

# query all points in a radius around the first 100 points
indices = kdtree.query_radius(data[:100], r=0.5, return_sorted=True)

(1000000, 3)
